In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
-- SELECT * from FIRE.PUBLIC."fire_df_final"
SELECT * from FIRE.PUBLIC."FIRE_LONG_LAT"

In [ ]:
fire_df = load_df.to_pandas()
fire_df

In [ ]:
invalid_counties = ["MEXICO", "STATE OF NEVADA", "STATE OF OREGON"] #not counties in ca
fire_df = fire_df[~fire_df["INCIDENT_COUNTY"].isin(invalid_counties)]
print(len(fire_df))

In [ ]:
missing_data_rows = fire_df[fire_df.isnull().any(axis=1)]
missing_data_rows

In [ ]:
fire_df.fillna(0, inplace=True)

In [ ]:
print(fire_df.columns)

In [ ]:
county_col = 'INCIDENT_COUNTY'

selected_features = ['AVG_ACRES_BURNED', 'AVG_TIME_TO_EXTINGUISH', 'AVG_CREWSINVOLVED', 'STATION_COUNT']
# features_to_normalize = fire_df.drop(columns=[county_col])
incident_county = fire_df[county_col] #save for later

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
normalized_features = scaler.fit_transform(fire_df[selected_features])
normalized_df = pd.DataFrame(normalized_features, columns=selected_features)
normalized_df

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, random_state=42)  # Adjust n_clusters based on your use case
normalized_df["cluster"] = kmeans.fit_predict(normalized_df)

# View clustering results
# Add clusters to the original DataFrame
fire_df["cluster"] = normalized_df["cluster"]

fire_df.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Count number of counties per cluster
cluster_counts = fire_df.groupby("cluster").size().reset_index(name="count")

# Plot
sns.barplot(x="cluster", y="count", data=cluster_counts)
plt.title("Number of Counties per Cluster")
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Plot pairplot of selected features colored by cluster labels
sns.pairplot(fire_df[selected_features + ['cluster']], hue='cluster', palette='Set2', markers=["o", "s", "D"])
plt.title("Pairplot of Features Colored by Clusters")
plt.show()


In [ ]:
from sklearn.metrics import silhouette_score

# Test multiple values of k
scores = []
k_values = range(2, 10)

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(normalized_features)
    score = silhouette_score(normalized_features, labels)
    scores.append(score)

# Plot the Silhouette Scores
plt.plot(k_values, scores, marker='o')
plt.title('Silhouette Score vs. Number of Clusters')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
plt.show()

In [ ]:
fire_df.columns

In [ ]:
# from sklearn.cluster import KMeans

# kmeans = KMeans(n_clusters=2, random_state=42)  # Adjust n_clusters based on your use case
# normalized_df["cluster"] = kmeans.fit_predict(normalized_features)

# # View clustering results
# normalized_df.head()

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, random_state=42)  # Adjust n_clusters based on your use case
normalized_df["cluster"] = kmeans.fit_predict(normalized_features)

# View clustering results
# Add clusters to the original DataFrame
fire_df["cluster"] = normalized_df["cluster"]

fire_df.head()

In [ ]:
# # cluster_stats = fire_df.groupby('cluster').agg(['mean', 'std', 'min', 'max'])
# # print(cluster_stats)
# print(fire_df.dtypes)
numeric_columns = fire_df.select_dtypes(include='number')
cluster_stats = fire_df.groupby('cluster')[numeric_columns.columns].agg(['mean', 'std', 'min', 'max'])
cluster_stats



In [ ]:
selected_features = ['AVG_ACRES_BURNED', 'AVG_TIME_TO_EXTINGUISH', 'AVG_CREWSINVOLVED', 'STATION_COUNT', 'cluster']
new_fire_df = fire_df[selected_features]
# print(selected_features)
numeric_columns = new_fire_df.select_dtypes(include='number')
cluster_stats = new_fire_df.groupby('cluster')[numeric_columns.columns].agg(['mean'])
cluster_stats

notes
- interactive map that displays clusters
- work with two main clusters
- do decision, alt way is to train a model
- work with LLM's to see how they pan out on a map
- make a number of fires df